# Import Libraries

In [10]:
!pip3 install psycopg2-binary
!pip3 install psycopg

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [11]:
import pandas as pd

import numpy as np

import psycopg2

from psycopg2 import sql, extras

import json

# Abstract Syntax Trees
import ast

from ast import literal_eval

# Global Params

In [71]:
DATA_ROOT='/home/recoil/shared_space/' # note to use recoil as the common uesr

# Database connection parameters
conn_params = {
    'dbname': 'intermodal_db',
    'user': 'jerry',
    'password': 'password2024JTS',
    'host': 'recoil.ise.utk.edu',
    'port': '5432'
}



# Util functions

In [72]:
def execute_sql_script_creation(conn_params, sql_script):
    try:
        ## connection variable
        conn = psycopg2.connect(**conn_params)
        # Create a cursor object using the connection
        cursor = conn.cursor()
        # Execute the SQL script
        cursor.execute(sql_script)
        # Commit the transaction
        conn.commit()
        print("SQL script executed successfully.")
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error executing SQL script: {error}")
    finally:
        # Close database connections
        if conn is not None:
            conn.close()
            print('Database connection closed.')


def execute_sql_script_insert(conn_params,sql_script,data):
    
    # Define the SQL query for bulk insert
    insert_query = sql.SQL(sql_script)
        
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(**conn_params)
        cursor = conn.cursor()
    
        # Create the table if not exists
        # cursor.execute(create_table_query)
    
        # Convert DataFrame to list of tuples
        data_tuples = [tuple(row) for row in data.to_numpy()]
    
        # Perform bulk insert
        extras.execute_values(cursor, insert_query, data_tuples)
    
        # Commit the transaction
        conn.commit()
    
        print("Bulk insert successful")
    except Exception as e:
        print(f"Error: {e}")
        if conn:
            conn.rollback()
    finally:
        # Close cursor and connection
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Prepare the db

In [73]:
"""
- We need to create the db first

- We use "intermodal_db" as the db name

- scripts log: password2024JTS vector_db
- docker exec -it postsgres bash
- psql -U jerry -d vector_db
- CREATE DATABASE intermodal_db;
 
- now we have the db

"""

'\n- We need to create the db first\n\n- We use "intermodal_db" as the db name\n\n- scripts log: password2024JTS vector_db\n- docker exec -it postsgres bash\n- psql -U jerry -d vector_db\n- CREATE DATABASE intermodal_db;\n \n- now we have the db\n\n'

# Create the schema

In [76]:
create_schema_sql = """

--
-- PostgreSQL database dump
--

-- Dumped from database version 16.3 (Debian 16.3-1.pgdg120+1)
-- Dumped by pg_dump version 16.4

-- Started on 2024-09-18 12:13:56 EDT

SET statement_timeout = 0;
SET lock_timeout = 0;
SET idle_in_transaction_session_timeout = 0;
SET client_encoding = 'UTF8';
SET standard_conforming_strings = on;
SELECT pg_catalog.set_config('search_path', '', false);
SET check_function_bodies = false;
SET xmloption = content;
SET client_min_messages = warning;
SET row_security = off;

SET default_tablespace = '';

SET default_table_access_method = heap;



--
-- TOC entry 218 (class 1259 OID 16400)
-- Name: dta_crd; Type: TABLE; Schema: public; Owner: jerry
--

CREATE TABLE public.dta_rtg (
    id SERIAL PRIMARY KEY,
    rtg_dms_mode VARCHAR(50) NOT NULL,
    rtg_dms_orig_string VARCHAR(255) NOT NULL,
    rtg_dms_dest_string VARCHAR(255) NOT NULL,
    rtg_distance_miles FLOAT NOT NULL,
    rtg_path TEXT NOT NULL,
    rtg_granularity_level VARCHAR(255) NOT NULL,
    rtg_type VARCHAR(50) NOT NULL
);

ALTER TABLE public.dta_rtg OWNER TO jerry;


CREATE SEQUENCE public.dat_rtg_id_seq
    AS integer
    START WITH 1
    INCREMENT BY 1
    NO MINVALUE
    NO MAXVALUE
    CACHE 1;


ALTER SEQUENCE public.dat_rtg_id_seq OWNER TO jerry;




CREATE TABLE public.dta_crd (
    id integer NOT NULL,
    crd_dms text,
    crd_latitude numeric,
    crd_longitude numeric,
    crd_dms_mode text,
    crd_granularity_level text
);



ALTER TABLE public.dta_crd OWNER TO jerry;

--
-- TOC entry 217 (class 1259 OID 16399)
-- Name: dta_crd_id_seq; Type: SEQUENCE; Schema: public; Owner: jerry
--

CREATE SEQUENCE public.dta_crd_id_seq
    AS integer
    START WITH 1
    INCREMENT BY 1
    NO MINVALUE
    NO MAXVALUE
    CACHE 1;


ALTER SEQUENCE public.dta_crd_id_seq OWNER TO jerry;

--
-- TOC entry 3386 (class 0 OID 0)
-- Dependencies: 217
-- Name: dta_crd_id_seq; Type: SEQUENCE OWNED BY; Schema: public; Owner: jerry
--

ALTER SEQUENCE public.dta_crd_id_seq OWNED BY public.dta_crd.id;


--
-- TOC entry 216 (class 1259 OID 16391)
-- Name: dta_dmd; Type: TABLE; Schema: public; Owner: jerry
--

CREATE TABLE public.dta_dmd (
    id integer NOT NULL,
    dmd_dms_mode character varying(50),
    dmd_dms_orig character varying(255),
    dmd_dms_dest character varying(255),
    dmd_granularity_level character varying(50),
    dmd_freight double precision,
    dmd_timestamp date
);


ALTER TABLE public.dta_dmd OWNER TO jerry;

--
-- TOC entry 215 (class 1259 OID 16390)
-- Name: dta_dmd_id_seq; Type: SEQUENCE; Schema: public; Owner: jerry
--

CREATE SEQUENCE public.dta_dmd_id_seq
    AS integer
    START WITH 1
    INCREMENT BY 1
    NO MINVALUE
    NO MAXVALUE
    CACHE 1;


ALTER SEQUENCE public.dta_dmd_id_seq OWNER TO jerry;

--
-- TOC entry 3387 (class 0 OID 0)
-- Dependencies: 215
-- Name: dta_dmd_id_seq; Type: SEQUENCE OWNED BY; Schema: public; Owner: jerry
--

ALTER SEQUENCE public.dta_dmd_id_seq OWNED BY public.dta_dmd.id;


--
-- TOC entry 222 (class 1259 OID 16418)
-- Name: dta_lca; Type: TABLE; Schema: public; Owner: jerry
--

CREATE TABLE public.dta_lca (
    id integer NOT NULL,
    lca_vehicle_type character varying(255),
    lca_transportation_mode character varying(255),
    lca_fuel_type character varying(255),
    lca_co_2_emission double precision,
    lca_emission_unit character varying(255)
);


ALTER TABLE public.dta_lca OWNER TO jerry;

--
-- TOC entry 221 (class 1259 OID 16417)
-- Name: dta_lca_id_seq; Type: SEQUENCE; Schema: public; Owner: jerry
--

CREATE SEQUENCE public.dta_lca_id_seq
    AS integer
    START WITH 1
    INCREMENT BY 1
    NO MINVALUE
    NO MAXVALUE
    CACHE 1;


ALTER SEQUENCE public.dta_lca_id_seq OWNER TO jerry;

--
-- TOC entry 3388 (class 0 OID 0)
-- Dependencies: 221
-- Name: dta_lca_id_seq; Type: SEQUENCE OWNED BY; Schema: public; Owner: jerry
--

ALTER SEQUENCE public.dta_lca_id_seq OWNED BY public.dta_lca.id;


--
-- TOC entry 220 (class 1259 OID 16409)
-- Name: dta_rts; Type: TABLE; Schema: public; Owner: jerry
--

CREATE TABLE public.dta_rts (
    id integer NOT NULL,
    rts_dms_mode text,
    rts_dms_string text,
    rts_reachable_cities text,
    rts_granularity_level text
);


ALTER TABLE public.dta_rts OWNER TO jerry;

--
-- TOC entry 219 (class 1259 OID 16408)
-- Name: dta_rts_id_seq; Type: SEQUENCE; Schema: public; Owner: jerry
--

CREATE SEQUENCE public.dta_rts_id_seq
    AS integer
    START WITH 1
    INCREMENT BY 1
    NO MINVALUE
    NO MAXVALUE
    CACHE 1;


ALTER SEQUENCE public.dta_rts_id_seq OWNER TO jerry;

--
-- TOC entry 3389 (class 0 OID 0)
-- Dependencies: 219
-- Name: dta_rts_id_seq; Type: SEQUENCE OWNED BY; Schema: public; Owner: jerry
--

ALTER SEQUENCE public.dta_rts_id_seq OWNED BY public.dta_rts.id;




--
-- TOC entry 3219 (class 2604 OID 16403)
-- Name: dta_crd id; Type: DEFAULT; Schema: public; Owner: jerry
--

ALTER TABLE ONLY public.dta_rtg ALTER COLUMN id SET DEFAULT nextval('public.dat_rtg_id_seq'::regclass);




--
-- TOC entry 3219 (class 2604 OID 16403)
-- Name: dta_crd id; Type: DEFAULT; Schema: public; Owner: jerry
--

ALTER TABLE ONLY public.dta_crd ALTER COLUMN id SET DEFAULT nextval('public.dta_crd_id_seq'::regclass);


--
-- TOC entry 3218 (class 2604 OID 16394)
-- Name: dta_dmd id; Type: DEFAULT; Schema: public; Owner: jerry
--

ALTER TABLE ONLY public.dta_dmd ALTER COLUMN id SET DEFAULT nextval('public.dta_dmd_id_seq'::regclass);


--
-- TOC entry 3221 (class 2604 OID 16421)
-- Name: dta_lca id; Type: DEFAULT; Schema: public; Owner: jerry
--

ALTER TABLE ONLY public.dta_lca ALTER COLUMN id SET DEFAULT nextval('public.dta_lca_id_seq'::regclass);


--
-- TOC entry 3220 (class 2604 OID 16412)
-- Name: dta_rts id; Type: DEFAULT; Schema: public; Owner: jerry
--

ALTER TABLE ONLY public.dta_rts ALTER COLUMN id SET DEFAULT nextval('public.dta_rts_id_seq'::regclass);

"""

In [77]:
# Call the function to execute the modified SQL script
# Run once only
execute_sql_script_creation(conn_params, create_schema_sql)

SQL script executed successfully.
Database connection closed.


## To Reset DB

In [74]:

# SQL block to drop tables excluding those required by PostGIS
drop_tables_sql = """
DO $$ 
DECLARE
    r RECORD;
    s RECORD;
BEGIN
    -- Loop through each table in the current schema
    FOR r IN (SELECT tablename FROM pg_tables WHERE schemaname = 'public' AND tablename NOT IN ('spatial_ref_sys'))
    LOOP
        -- Generate and execute the drop table command
        EXECUTE 'DROP TABLE IF EXISTS ' || quote_ident(r.tablename) || ' CASCADE';
    END LOOP;

    -- Loop through each sequence in the current schema
    FOR s IN (SELECT sequence_name FROM information_schema.sequences WHERE sequence_schema = 'public')
    LOOP
        -- Generate and execute the drop sequence command
        EXECUTE 'DROP SEQUENCE IF EXISTS ' || quote_ident(s.sequence_name) || ' CASCADE';
    END LOOP;
END $$;

"""


In [75]:
# if to reset !WARNING
execute_sql_script(conn_params, drop_tables_sql)

SQL script executed successfully.
Database connection closed.


# Load Intermodal data

## DMD

In [78]:
## Demands dataset it comes from the FAF dataset.
demands_df=pd.read_csv(DATA_ROOT+'recoil_data/intermodal_output/new_intermodal_data.csv').iloc[:,:-2]

# Renaming columns directly
new_column_names = [
    'DMD_dms_mode', 'DMD_dms_orig', 'DMD_dms_dest',
    'DMD_tons_2017', 'DMD_tons_2018', 'DMD_tons_2019', 'DMD_tons_2020',
    'DMD_tons_2021', 'DMD_tons_2022', 'DMD_tons_2025', 'DMD_tons_2030',
    'DMD_tons_2035', 'DMD_tons_2040', 'DMD_tons_2045', 'DMD_tons_2050'
]

# Assign new column names to the DataFrame
demands_df.columns = new_column_names
demands_df

,DMD_dms_mode,DMD_dms_orig,DMD_dms_dest,DMD_tons_2017,DMD_tons_2018,DMD_tons_2019,DMD_tons_2020,DMD_tons_2021,DMD_tons_2022,DMD_tons_2025,DMD_tons_2030,DMD_tons_2035,DMD_tons_2040,DMD_tons_2045,DMD_tons_2050
0,Rail,Albany NY CFS Area,Albany NY CFS Area,2.151089,2.129259,2.029196,1.916540,2.038797,1.878167,3.134651,3.790910,4.533992,5.379749,6.368823,7.463871
1,Rail,Albany NY CFS Area,Buffalo NY CFS Area,3.284010,3.520622,3.284021,2.672125,3.044126,3.221475,2.695342,2.512051,2.331322,2.156351,1.989575,1.821083
2,Rail,Albany NY CFS Area,Chicago IL-IN-WI (IN Part),6.951653,7.110767,6.815517,6.812078,6.667921,6.491779,6.441755,6.518915,6.411717,6.588118,6.855076,7.105745
3,Rail,Albany NY CFS Area,Los Angeles CA,5.005863,5.120441,4.907832,4.905355,4.801548,4.674710,5.197703,5.403830,5.448775,5.732618,6.135413,6.496954
4,Rail,Albany NY CFS Area,New Orleans LA-MS (LA Part),73.983521,73.092394,73.074641,73.282022,87.103521,74.473165,62.204445,63.235298,64.843910,66.112679,67.183281,68.491730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8169,Water,Tulsa OK,Memphis TN-MS-AR (TN Part),59.184204,71.027237,78.699879,67.616428,62.414230,66.359019,56.475910,47.302441,39.377861,32.520535,28.265242,25.434513
8170,Water,Tulsa OK,New Orleans LA-MS (LA Part),3186.414521,2825.703762,3143.528865,2985.335631,3218.950326,2306.014007,3621.027210,4053.153118,4476.015799,4880.257749,5236.597011,5580.831539
8171,Water,Tulsa OK,St. Louis MO-IL (IL Part),155.417084,186.516760,206.665037,177.560013,163.899099,174.258071,199.778180,201.149440,202.848750,202.327790,175.853320,158.241820
8172,Water,Tulsa OK,Tulsa OK,738.261017,639.612765,742.298501,706.657789,765.881157,529.476010,714.155880,750.377040,790.618040,829.318880,871.874420,908.602350


In [79]:
# Melting the DataFrame to transform the tons columns into rows for better vizualization
df_melted = demands_df.melt(id_vars=['DMD_dms_mode', 'DMD_dms_orig', 'DMD_dms_dest'], var_name='DMD_granularity_level',value_name='DMD_freight')

# Extracting the year from the 'type' column and creating a timestamp
df_melted['DMD_timestamp'] = df_melted['DMD_granularity_level'].str.extract('(\d{4})').astype(int)
df_melted['DMD_timestamp'] = pd.to_datetime(df_melted['DMD_timestamp'], format='%Y')

# Cleaning up the 'type' column to only contain 'DMD_freight'
df_melted['DMD_granularity_level'] = 'Hubs'


In [80]:
demands_df=df_melted
del df_melted

In [81]:
demands_df

,DMD_dms_mode,DMD_dms_orig,DMD_dms_dest,DMD_granularity_level,DMD_freight,DMD_timestamp
0,Rail,Albany NY CFS Area,Albany NY CFS Area,Hubs,2.151089,2017-01-01
1,Rail,Albany NY CFS Area,Buffalo NY CFS Area,Hubs,3.284010,2017-01-01
2,Rail,Albany NY CFS Area,Chicago IL-IN-WI (IN Part),Hubs,6.951653,2017-01-01
3,Rail,Albany NY CFS Area,Los Angeles CA,Hubs,5.005863,2017-01-01
4,Rail,Albany NY CFS Area,New Orleans LA-MS (LA Part),Hubs,73.983521,2017-01-01
...,...,...,...,...,...,...
98083,Water,Tulsa OK,Memphis TN-MS-AR (TN Part),Hubs,25.434513,2050-01-01
98084,Water,Tulsa OK,New Orleans LA-MS (LA Part),Hubs,5580.831539,2050-01-01
98085,Water,Tulsa OK,St. Louis MO-IL (IL Part),Hubs,158.241820,2050-01-01
98086,Water,Tulsa OK,Tulsa OK,Hubs,908.602350,2050-01-01


In [82]:
execute_sql_script_insert(conn_params,"""INSERT INTO DTA_DMD (DMD_dms_mode, DMD_dms_orig, DMD_dms_dest, DMD_granularity_level, DMD_freight, DMD_timestamp) VALUES %s""",demands_df)

Bulk insert successful


## CRD

In [83]:

# Load the original master cities data
master_df = pd.read_csv(DATA_ROOT+'recoil_data/FAF/MASTER_CITIES.csv').iloc[:, :-1]

# Assign new column names to the DataFrame
new_column_names = [
    'CRD_dms_orig', 'CRD_dms_dest', 'CRD_dms_mode', 'CRD_latitude_dmsorig', 'CRD_longitude_dmsorig', 'CRD_latitude_dmsdest', 'CRD_longitude_dmsdest'
]
master_df.columns = new_column_names

# Get the list of unique cities from both origin and destination
orig_dest_cities = pd.DataFrame({
    'City': list(set(master_df['CRD_dms_orig']).union(set(master_df['CRD_dms_dest']))),
})

# Merging the lat/long for origin cities with their modes
orig_data = master_df[['CRD_dms_orig', 'CRD_latitude_dmsorig', 'CRD_longitude_dmsorig', 'CRD_dms_mode']].drop_duplicates().rename(
    columns={
        'CRD_dms_orig': 'City', 
        'CRD_latitude_dmsorig': 'Latitude', 
        'CRD_longitude_dmsorig': 'Longitude',
        'CRD_dms_mode': 'Mode'
    })

# Merging the lat/long for destination cities with their modes
dest_data = master_df[['CRD_dms_dest', 'CRD_latitude_dmsdest', 'CRD_longitude_dmsdest', 'CRD_dms_mode']].drop_duplicates().rename(
    columns={
        'CRD_dms_dest': 'City', 
        'CRD_latitude_dmsdest': 'Latitude', 
        'CRD_longitude_dmsdest': 'Longitude',
        'CRD_dms_mode': 'Mode'
    })

# Concatenating the origin and destination data with their respective modes
all_cities_data = pd.concat([orig_data, dest_data]).drop_duplicates()

# Merging with the full list of cities
result = pd.merge(orig_dest_cities, all_cities_data, on='City', how='left')

# Adding 'CRD_granularity_level' and renaming the columns
result['CRD_granularity_level'] = 'Hubs'

# Renaming columns to match the final structure
new_column_names = [
    'CRD_dms', 'CRD_latitude', 'CRD_longitude', 'CRD_dms_mode', 'CRD_granularity_level'
]
result.columns = new_column_names

result

,CRD_dms,CRD_latitude,CRD_longitude,CRD_dms_mode,CRD_granularity_level
0,Washington,38.895037,-77.036543,road,Hubs
1,Washington,38.895037,-77.036543,rail,Hubs
2,Tucson AZ,32.222876,-110.974847,road,Hubs
3,Tucson AZ,32.222876,-110.974847,rail,Hubs
4,Charleston WV,38.357611,-81.639517,rail,Hubs
...,...,...,...,...,...
234,Atlanta GA,33.748992,-84.390264,road,Hubs
235,El Paso TX,31.755051,-106.488234,road,Hubs
236,El Paso TX,31.755051,-106.488234,rail,Hubs
237,Fresno CA,36.738324,-119.782872,road,Hubs


In [84]:
execute_sql_script_insert(conn_params,"""INSERT INTO DTA_CRD (CRD_dms, CRD_latitude, CRD_longitude, CRD_dms_mode, CRD_granularity_level) VALUES %s""",result)

Bulk insert successful


## RTS

In [85]:
routes_df=pd.read_csv(DATA_ROOT+'recoil_data/intermodal_output/routes_paths.csv',index_col='Unnamed: 0')[['dms_mode', 'dms_string', 'reachable_cities']]
print(routes_df.columns)
new_column_names = [
    'RTS_dms_mode','RTS_dms_string', 'RTS_reachable_cities'
    # , 'RTS_paths', 'RTS_distances'
    
]

# Assign new column names to the DataFrame
routes_df.columns = new_column_names



Index(['dms_mode', 'dms_string', 'reachable_cities'], dtype='object')


In [86]:
import ast
# Parse the list strings into actual lists using ast.literal_eval
routes_df['RTS_reachable_cities'] = routes_df['RTS_reachable_cities'].apply(ast.literal_eval)
# routes_df = routes_df.explode('RTS_reachable_cities')


In [87]:
# Expand the lists into separate rows
routes_df = routes_df.explode('RTS_reachable_cities')
routes_df['RTS_granularity_level']='Hubs'

In [88]:
routes_df

,RTS_dms_mode,RTS_dms_string,RTS_reachable_cities,RTS_granularity_level
0,Road,Albany NY CFS Area,New York NY-NJ-CT-PA (NY Part),Hubs
0,Road,Albany NY CFS Area,Rochester NY,Hubs
1,Road,Atlanta GA,Birmingham AL,Hubs
1,Road,Atlanta GA,Jacksonville FL-GA CFS Area (FL Part),Hubs
1,Road,Atlanta GA,Greenville SC,Hubs
...,...,...,...,...
84,Rail,Houston TX,Beaumont TX,Hubs
85,Rail,Milwaukee WI,Chicago IL-IN-WI (IL Part),Hubs
85,Rail,Milwaukee WI,Iowa,Hubs
86,Rail,Minneapolis-St. Paul MN-WI (MN Part),Iowa,Hubs


In [89]:
execute_sql_script_insert(conn_params,"""INSERT INTO DTA_RTS (RTS_dms_mode,RTS_dms_string,RTS_reachable_cities,RTS_granularity_level) VALUES %s""",routes_df)



Bulk insert successful


## LCA

In [90]:
lca_df = pd.read_excel(DATA_ROOT+'RECOIL_Data_Collection/LCA/Emission  Rates.xlsx')
lca_df.replace({'Marine': 'Water'}, inplace=True)
lca_df.loc[1, 'Fuel Type'] = np.nan
lca_df = lca_df.fillna('Any')

new_column_names = [
    'LCA_Vehicle_Type','LCA_Transportation_Mode','LCA_Fuel_Type','LCA_CO_2_Emission','LCA_Emission_Unit',
]

# Assign new column names to the DataFrame
lca_df.columns = new_column_names

lca_df

,LCA_Vehicle_Type,LCA_Transportation_Mode,LCA_Fuel_Type,LCA_CO_2_Emission,LCA_Emission_Unit
0,Long_Haul Trucks,Road,Conventional and LS Diesel,1583.049300,g/miles
1,Long_Haul Trucks,Road,Any,1317.904932,g/miles
2,Long_Haul Trucks,Road,LNG,1284.573213,g/miles
3,Long_Haul Trucks,Road,Electricity,1171.895974,g/miles
4,Short_Haul Trucks,Road,Conventional and LS Diesel,2790.450436,g/miles
5,Short_Haul Trucks,Road,CNG,2322.943742,g/miles
6,Short_Haul Trucks,Road,LNG,2409.875791,g/miles
7,Short_Haul Trucks,Road,Electricity,1096.908323,g/miles
8,Freight,Rail,Diesel,24.777712,g/ton. miles
9,Freight,Rail,LNG,19.373010,g/ton. miles


In [91]:
execute_sql_script_insert(conn_params, """INSERT INTO DTA_LCA (LCA_Vehicle_Type, LCA_Transportation_Mode, LCA_Fuel_Type, LCA_CO_2_Emission, LCA_Emission_Unit) VALUES %s""", lca_df)


Bulk insert successful


# RTG

In [95]:
rtg_df = pd.read_csv(DATA_ROOT+'shared_space/recoil_data/intermodal_output/allpaths_df.csv',index_col='Unnamed: 0')

In [99]:
# Add missing columns with default values
rtg_df['rtg_granularity_level'] = 'Hubs'  # Example value, adjust as needed
rtg_df['rtg_type'] = 'pc'  # Example value, adjust as needed


In [100]:
rtg_df

,dms_mode,dms_orig_string,dms_dest_string,distance_miles,path,rtg_granularity_level,rtg_type
0,Road,Cleveland OH,Nashville TN,509.179484,"[(-81.69369399965134, 41.49966399976782), (-81...",Hubs,pc
1,Road,Cleveland OH,New Mexico,1524.149477,"[(-81.69369399965134, 41.49966399976782), (-81...",Hubs,pc
2,Road,Cleveland OH,New Orleans LA-MS (LA Part),1028.635797,"[(-81.69369399965134, 41.49966399976782), (-81...",Hubs,pc
3,Road,Cleveland OH,New York NY-NJ-CT-PA (CT Part),449.163446,"[(-81.69369399965134, 41.49966399976782), (-81...",Hubs,pc
4,Road,Cleveland OH,New York NY-NJ-CT-PA (NJ Part),433.294682,"[(-81.69369399965134, 41.49966399976782), (-81...",Hubs,pc
...,...,...,...,...,...,...,...
8169,Rail,Rochester NY,Rochester NY,0.000000,"[(-77.6307472125819, 43.224141412705066)]",Hubs,pc
8170,Rail,Rochester NY,Salt Lake City UT,2057.293812,"[(-77.6307472125819, 43.224141412705066), (-77...",Hubs,pc
8171,Rail,Rochester NY,San Francisco CA,2873.431097,"[(-77.6307472125819, 43.224141412705066), (-77...",Hubs,pc
8172,Rail,Sacramento CA,Jacksonville FL-GA CFS Area (FL Part),2856.323161,"[(-121.47876347635706, 38.58614737057163), (-1...",Hubs,pc


In [102]:
execute_sql_script_insert(conn_params, """INSERT INTO DTA_RTG (rtg_dms_mode, rtg_dms_orig_string, rtg_dms_dest_string, rtg_distance_miles, rtg_path, rtg_granularity_level, rtg_type) VALUES %s""", rtg_df)

Bulk insert successful
